In [3]:
# import nltk

# nltk.download('punkt')
# nltk.download('stopwords')

In [5]:
# !pip install gensim

In [3]:
# from google.colab import files
# files.upload()

In [8]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel

import numpy as np
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from numpy import linalg as LA
from sklearn.decomposition import TruncatedSVD
import math

In [9]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head(2)

(743, 43)


,X.AUTHID,anger_doc_sum,anticipation_doc_sum,arousal_doc_sum,disgust_doc_sum,dominance_doc_sum,fear_doc_sum,joy_doc_sum,neg_doc_sum,num_text_words,...,cSIN_label_fin,cEXC_label_fin,cCOM_label_fin,cRUG_label_fin,cSOP_label_fin,cSIN_fin_rank,cEXC_fin_rank,cCOM_fin_rank,cRUG_fin_rank,cSOP_fin_rank
0,b'http://blog.searsholdings.com/leadership-vie...,1,9,31.317,0,38.293,7,16,7,124,...,n,y,y,n,y,5.0,1.0,2.0,3.0,4.0
1,b'https://atyourservice.blogs.xerox.com/catego...,0,4,23.260,0,32.665,0,1,2,126,...,y,n,y,n,y,NaN,NaN,NaN,NaN,NaN


In [10]:
site_content = data['site.content']
docs = {}
for i in range(site_content.size):
    docs[i] = eval(site_content[i]).decode('utf-8')

In [11]:
original_corpus = [1]*len(docs)
corpus = [1]*len(docs)
copy_corpus = [1]*len(docs)

for i in range(len(docs)):
    original_corpus[i] = sent_tokenize(docs[i])
    corpus[i] = sent_tokenize(docs[i])
    copy_corpus[i] = sent_tokenize(docs[i])

In [12]:
# print(site_content[20])
# print(docs[20])
# print(corpus[20])

b"Join me! Ill be hosting a unique set of customersat ;, SYN109, (Wednesday May 24, 4:45pm) where well have a discussion and audience-driven Q&A with 4 Citrix Cloud customers:;Well cut the marketing fluff. No speeds and feeds. Just candid, matter-of-fact discussions about:;Ken is Sr. Director of Product Marketing in Citrix's Cloud Services group, and spends most of his time helping lead the charge to converge concepts of cloud computing, virtual workspaces, and hosted service providers. You can also find Ken on Twitter as @fountnhead; Weve seen increasing traction for NetScaler in customer deployments as they migrate to the cloud. One of the key differentiators ; Citrix VP of WW Sales and Strategy, Ash Vijayakanthan shares his thoughts on the benefits Citrix Cloud brings for Citrix Service Providers ; NetScaler Management and Analytics from Citrix allows administrators to easily view, automate, and manage all their application infrastructure, as well ;Dont worry, you can unsubscribe at

In [13]:
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()
remove_punctuation_map = dict((ord(char), ' ') for char in string.punctuation)
integer_chars = [str(digit) for digit in list(range(10))]
remove_integer_map = dict((ord(char), None) for char in integer_chars)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def lemm_tokens(tokens):
    return [lemmatizer.lemmatize(item) for item in tokens]

def filtersent(text):
    # remove puctuationas and lower the case
    simpletext = text.lower().translate(remove_punctuation_map).translate(remove_integer_map)
    # tokenize
    words = nltk.word_tokenize(simpletext)
    # remove stop words
    words = [w for w in words if w not in stopwords.words('english')]
    # stem them
    words = lemm_tokens(words)
#     words = stem_tokens(words)
    words = [w for w in words if w not in string.ascii_lowercase]
    return TreebankWordDetokenizer().detokenize(words)

In [14]:
for i in range(len(corpus)):
    for j in range(len(corpus[i])):
        corpus[i][j] = filtersent(original_corpus[i][j])
        copy_corpus[i][j] = original_corpus[i][j]

In [15]:
for i in range(len(corpus)):
    idx = []
    for j in range(len(corpus[i])):
        if(corpus[i][j] == ''):
            idx.insert(0,j)
    for index in idx:
        del corpus[i][index]
        del copy_corpus[i][index]

In [16]:
# model = LsiModel(common_corpus, id2word=common_dictionary)
# vectorized_corpus = model[common_corpus]
# print(corpus[498])
# print(original_corpus[498])

['sure heard lifeblood engine', 'choose correct oil could accelerate engine wear cause premature failure', 'right help fuel economy emission', 'choosing right oil vehicle critical durability efficiency lifespan', 'regardless type vehicle buying oil package', 'icon magazine designates product tested certified american petroleum institute api', 'additionally superior oil performance viscosity energy conservation', 'check find repair shop insurance work one', 'unable find agent zip code', 'please enter new zip code visit allstate agent locator', 'according viscosity property resistance flow fluid semifluid', 'regarding oil fancy way saying viscosity measure well oil flow different condition', 'motor oil viscosity represented numerical value', 'designated motor oil grade signify oil rating sae', 'climate two part viscosity number', 'first number letter representation well oil flow colder weather', 'stand winter', 'lower number better oil flow colder temperature', 'second number measure oil

In [17]:
k = 3

In [18]:
top_corpus = [1]*len(corpus)

# Gong Liu
for s in range(len(corpus)):
# if(True):
#     s = 0
    
    no_sent = len(corpus[s])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus[s])
    word_matrix = np.array(X.toarray())
    
    U, S, Vh = np.linalg.svd(word_matrix.T, full_matrices=False)
    
    assert np.allclose(word_matrix.T, U.dot(np.diag(S).dot(Vh))), "Something Wong!"
    
    select_sents = []
    for i in range(k):
        sorted_info = np.argsort(Vh[i,:])
        for j in range(no_sent):
            if sorted_info[no_sent-j-1] in select_sents:
                continue
            else:
                select_sents.append(sorted_info[no_sent-j-1])
                break
    
    select_sents = np.sort(select_sents)
    
    top_sents = []
    for i in range(k):
        top_sents.append(copy_corpus[s][select_sents[i]])
            
    top_corpus[s] = top_sents

In [19]:
# top_corpus[0]
print(np.allclose(word_matrix.T, U.dot(np.diag(S).dot(Vh))))

True


In [20]:
top_data = pd.DataFrame({"X.AUTHID": data["X.AUTHID"],
                        "LSA_GL_1": [ s[0] for s in top_corpus],
                         "LSA_GL_2": [ s[1] for s in top_corpus],
                         "LSA_GL_3": [ s[2] for s in top_corpus]})
top_data.head(5)

,X.AUTHID,LSA_GL_1,LSA_GL_2,LSA_GL_3
0,b'http://blog.searsholdings.com/leadership-vie...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...
1,b'https://atyourservice.blogs.xerox.com/catego...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,"; Topics: ;, ;, ;, ;, ;, ;, ;"
3,b'https://connect.blogs.xerox.com/author/lauri...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,All rights reserved.
4,b'http://blog.searsholdings.com/2016/07/page/2/',SHC Speaks by As the retail landscape continue...,"With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...


In [21]:
top_corpus = [1]*len(corpus)

# steinberger and Jezek
for s in range(len(corpus)):
# if(True):
#     s = 2
    
    no_sent = len(corpus[s])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus[s])
    word_matrix = np.array(X.toarray())
    
    U, S, Vh = np.linalg.svd(word_matrix.T, full_matrices=False)

    Vh = Vh*(Vh>=0)
    
    length_sent = S.dot(Vh)
    idx_top_sent = np.argsort(length_sent)
    
    select_sents = np.zeros(no_sent)
    for i in range(k):
        select_sents[idx_top_sent[no_sent-i-1]] = 1
    
#     print(select_sents, idx_top_sent)
    top_sents = []
    for i in range(no_sent):
        if(select_sents[i]==1):
            top_sents.append(copy_corpus[s][i])
            
    top_corpus[s] = top_sents

In [22]:
# count matrix
count_word = [1]*len(corpus)
count_matrix = [1]*len(corpus)
count_persent = [1]*len(corpus) # no of words per sentence

vectorizer = CountVectorizer()
for i in range(len(corpus)):
    count_word[i] = vectorizer.fit_transform(corpus[i])
    count_matrix[i] = count_word[i].toarray()
    count_persent[i] = np.sum(count_matrix[i], axis=1)

In [23]:
top_data["LSA_SJ_1"] = [s[0] for s in top_corpus]
top_data["LSA_SJ_2"] = [s[1] for s in top_corpus]
top_data["LSA_SJ_3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,LSA_GL_1,LSA_GL_2,LSA_GL_3,LSA_SJ_1,LSA_SJ_2,LSA_SJ_3
0,b'http://blog.searsholdings.com/leadership-vie...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...
1,b'https://atyourservice.blogs.xerox.com/catego...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,"; Topics: ;, ;, ;, ;, ;, ;, ;","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,All rights reserved.,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...
4,b'http://blog.searsholdings.com/2016/07/page/2/',SHC Speaks by As the retail landscape continue...,"With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...


In [24]:
top_corpus = [1]*len(corpus)
lam = 0.7
mu = 0.85

# steinberger and Jezek
for s in range(len(corpus)):
# if(True):
#     s = 2
    
    no_sent = len(corpus[s])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus[s])
    word_matrix = np.array(X.toarray())
    
    U, S, Vh = np.linalg.svd(word_matrix.T, full_matrices=False)

    Vh = Vh*(Vh>=0)
    
    length_sent = S.dot(Vh)
    
    # similarity matrix
    sim = np.zeros((no_sent, no_sent))
    worthless_sent = []
    for x in range(no_sent):
        for y in range(no_sent):
            adotb = np.dot(count_matrix[s][x], count_matrix[s][y])
            a = LA.norm(count_matrix[s][x])
            b = LA.norm(count_matrix[s][y])
            if(x==y):
                sim[x,y] = (1-mu)/no_sent
            else:
                sim[x,y] = mu*(adotb/(a*b)) + (1-mu)/no_sent
                
    denom = np.sum(sim,axis=1)
    for x in range(no_sent):
        sim[x,:] = sim[x,:] * 1/denom[x]
    
    select_sents = np.zeros(no_sent)
    for i in range(k):
        idx = np.argmax(length_sent)
        select_sents[idx] = 1
        length_sent[idx] = -np.inf
        length_sent = [lam*length_sent[i] - (1-lam)*sim[idx,i] for i in range(no_sent)]
    
#     print(select_sents, idx_top_sent)
    top_sents = []
    for i in range(no_sent):
        if(select_sents[i]==1):
            top_sents.append(copy_corpus[s][i])
            
    top_corpus[s] = top_sents

In [25]:
top_corpus[0]

['I was joined by dozens of SHC associates and family members who marched in the parade or cheered from the crowd.',
 'As a company that represents a true cross section of America, Sears Holdings gives more to our members no matter who they are or who they love.',
 'This is a credit to the hard work and effort of our PRIDE Associate Resource Group and all its members and partners.']

In [26]:
top_data["LSA_SJMMR_1"] = [s[0] for s in top_corpus]
top_data["LSA_SJMMR_2"] = [s[1] for s in top_corpus]
top_data["LSA_SJMMR_3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,LSA_GL_1,LSA_GL_2,LSA_GL_3,LSA_SJ_1,LSA_SJ_2,LSA_SJ_3,LSA_SJMMR_1,LSA_SJMMR_2,LSA_SJMMR_3
0,b'http://blog.searsholdings.com/leadership-vie...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...
1,b'https://atyourservice.blogs.xerox.com/catego...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,"; Topics: ;, ;, ;, ;, ;, ;, ;","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,All rights reserved.,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...
4,b'http://blog.searsholdings.com/2016/07/page/2/',SHC Speaks by As the retail landscape continue...,"With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...


In [27]:
top_corpus = [1]*len(corpus)
# 
# Murray et al
for s in range(len(corpus)):
# if(True):
#     s = 347
    
    no_sent = len(corpus[s])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus[s])
    word_matrix = np.array(X.toarray())
    
    U, S, Vh = np.linalg.svd(word_matrix.T, full_matrices=False)
    
    concept_scores = S[:k] * 1/(np.sum(S[:k])) 
    
    select_sents = []
    for i in range(k):
        if(len(select_sents)>=k):
            break
        X = np.argsort(Vh[i,:])
        j = 0
        x = 0
        while(j<=np.round(concept_scores[i]*k)):
            if(len(select_sents)>=k):
                break
            if X[no_sent-x-1] not in select_sents:
                select_sents.append(X[no_sent-x-1])
                j+=1
            x+=1
    
    select_sents = np.sort(select_sents)
    
    top_sents = []
    for i in range(k):
        top_sents.append(copy_corpus[s][select_sents[i]])
            
    top_corpus[s] = top_sents

In [28]:
top_corpus[347]

['Traditionally prepared by cooking Basmati rice with succulent chunks of chicken, cashews and raisins infused with an unmistakable blend of enticing spices, aromas and flavors, including a pinch of saffron salt.',
 '301 comments says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says says Get seasonal recipes, cooking tips, product highlights and coupons delivered to your inbox!',
 'Receive recipes news and tips.']

In [29]:
top_data["LSA_MEA_1"] = [s[0] for s in top_corpus]
top_data["LSA_MEA_2"] = [s[1] for s in top_corpus]
top_data["LSA_MEA_3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,LSA_GL_1,LSA_GL_2,LSA_GL_3,LSA_SJ_1,LSA_SJ_2,LSA_SJ_3,LSA_SJMMR_1,LSA_SJMMR_2,LSA_SJMMR_3,LSA_MEA_1,LSA_MEA_2,LSA_MEA_3
0,b'http://blog.searsholdings.com/leadership-vie...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...
1,b'https://atyourservice.blogs.xerox.com/catego...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,"; Topics: ;, ;, ;, ;, ;, ;, ;","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",This will reduce your risk of non-compliance t...,"; Topics: ;, ;, ;, ;, ;, ;, ;"
3,b'https://connect.blogs.xerox.com/author/lauri...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,All rights reserved.,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...
4,b'http://blog.searsholdings.com/2016/07/page/2/',SHC Speaks by As the retail landscape continue...,"With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,SHC Speaks by As the retail landscape continue...,"For our Danbury Sears store, this meant that w...","With the pride that I have for Sears Holdings,..."


In [30]:
top_corpus = [1]*len(corpus)

# cross method
for s in range(len(corpus)):
# if(True):
#     s = 2
    
    no_sent = len(corpus[s])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus[s])
    word_matrix = np.array(X.toarray())
    
    U, S, Vh = np.linalg.svd(word_matrix.T, full_matrices=False)

    assert np.prod([[np.abs(Vh[i][j]) <= 1.001 for i in range(no_sent)] for j in range(no_sent)])==1, print(Vh, [[np.abs(Vh[i][j]) <= 1 for i in range(no_sent)] for j in range(no_sent)])
#     print(Vh)
    Vh = Vh*(Vh>=np.mean(Vh, axis=0))*(Vh>=0)
#     print(Vh)
    
    length_sent = S.T.dot(Vh)
    idx_top_sent = np.argsort(length_sent)
    
    select_sents = np.zeros(no_sent)
    for i in range(k):
        select_sents[idx_top_sent[no_sent-i-1]] = 1
    
#     print(select_sents, idx_top_sent)
    top_sents = []
    for i in range(no_sent):
        j=0
        while(j<select_sents[i]):
            top_sents.append(copy_corpus[s][i])
            j+=1
            
    top_corpus[s] = top_sents

In [31]:
top_data["LSA_CM_1"] = [s[0] for s in top_corpus]
top_data["LSA_CM_2"] = [s[1] for s in top_corpus]
top_data["LSA_CM_3"] = [s[2] for s in top_corpus]
                     
top_data.head(5)

,X.AUTHID,LSA_GL_1,LSA_GL_2,LSA_GL_3,LSA_SJ_1,LSA_SJ_2,LSA_SJ_3,LSA_SJMMR_1,LSA_SJMMR_2,LSA_SJMMR_3,LSA_MEA_1,LSA_MEA_2,LSA_MEA_3,LSA_CM_1,LSA_CM_2,LSA_CM_3
0,b'http://blog.searsholdings.com/leadership-vie...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...,As a company that represents a true cross sect...,"In addition, I also proud to report that out o...",This is a credit to the hard work and effort o...,I was joined by dozens of SHC associates and f...,As a company that represents a true cross sect...,This is a credit to the hard work and effort o...
1,b'https://atyourservice.blogs.xerox.com/catego...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...,The ConnectKey for SharePoint helps you share ...,All rights reserved.,Xerox and Xerox and Design are trademarks of X...,Recently new support videos were posted to the...,The ConnectKey for SharePoint helps you share ...,Xerox and Xerox and Design are trademarks of X...
2,b'http://blogs.zebra.com/whitepaper-label',"; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,"; Topics: ;, ;, ;, ;, ;, ;, ;","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how...","; March 17, 2015 at 8:03 AM ;Product identifi...",This will reduce your risk of non-compliance t...,"; Topics: ;, ;, ;, ;, ;, ;, ;","; March 17, 2015 at 8:03 AM ;Product identifi...",It is critical that the labels you utilize are...,";In this paper, Zebra provides guidance on how..."
3,b'https://connect.blogs.xerox.com/author/lauri...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,All rights reserved.,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Xerox and Xerox and Design are trademarks of X...,Writer and PR Consultant Email LinkedIn Twitte...,"By By Laurie Riedman, PR Consultant Throughout...",Remembering how when my children were young it...
4,b'http://blog.searsholdings.com/2016/07/page/2/',SHC Speaks by As the retail landscape continue...,"With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,"On Saturday, July 2, Sears celebrated the gran...","With the pride that I have for Sears Holdings,...",More than 20 Sears associates volunteered with...,SHC Speaks by As the retail landscape continue...,"For our Danbury Sears store, this meant that w...","With the pride that I have for Sears Holdings,...","On Saturday, July 2, Sears celebrated the gran...","With the pride that I 

In [32]:
length_sent

array([ 7.8841006 ,  8.12944372, 10.16695793, 12.85659764,  5.57100004,
        6.56141972,  5.18781402,  6.28412242, 12.99813482,  8.11978095,
        5.56523196,  8.72655051,  9.60958102,  6.69643147,  8.87892088,
        6.05764634,  8.22442853,  7.04346529,  8.54251867, 10.68166614,
        8.37364932,  7.55370024,  4.55600447,  6.26625888, 11.4738834 ,
        8.73063673,  3.70994106, 10.46550466,  8.1689398 ,  7.43029995,
        5.43784477, 11.51484301,  7.37180653,  8.19135795,  4.93610191,
       11.89664502,  7.3848299 ,  5.11369075,  7.49611898,  7.9132801 ,
        7.27779972,  8.27143162,  6.6826924 ,  7.7090774 ,  4.70214384,
        9.64530222, 49.69909061,  0.97015802,  8.53961928])

In [33]:
select_sents

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [34]:
S

array([49.69980716,  9.10052147,  6.50417541,  5.73704751,  5.40124845,
        5.05411878,  4.90748921,  4.71826842,  4.64757543,  4.59295064,
        4.43816285,  4.25719942,  4.19907407,  4.00175177,  3.9040669 ,
        3.81946442,  3.77104736,  3.75066766,  3.7054635 ,  3.60720127,
        3.50048436,  3.46711774,  3.32743415,  3.26058364,  3.24005075,
        3.14965089,  3.08509599,  3.05858549,  3.02512562,  2.95905031,
        2.90250018,  2.82130386,  2.80652648,  2.68776072,  2.64610796,
        2.62798559,  2.58291576,  2.49956669,  2.46862406,  2.45178793,
        2.3641905 ,  2.32785049,  2.2992847 ,  2.22997448,  2.08840617,
        2.07001547,  2.02492507,  1.80930209,  1.3869726 ])

In [35]:
np.sum(top_data["LSA_CM_1"] == top_data["LSA_SJ_1"])

681

In [36]:
top_data.to_csv('topdata_LSA.csv',encoding='utf-8', index=False)

In [35]:
np.argsort([4, 5, 2, 4])

array([2, 0, 3, 1])